# Using Open-source HuggingFace Models to Generate QAs from Raw Data

In this example, we will show you how to generate question-answers (QAs) from give text strings using open-source Huggingface models via uniflow's [HuggingFaceModelFlow](https://github.com/CambioML/uniflow/blob/main/uniflow/flow/model_flow.py#L86).

### Before running the code

You will need to `uniflow` conda environment to run this notebook. You can set up the environment following the instruction: https://github.com/CambioML/uniflow/tree/main#installation.

### Update system path

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

### Import dependency

In [ ]:
from dotenv import load_dotenv
from IPython.display import display

from uniflow.client import Client
from uniflow.flow.flow_factory import FlowFactory
from uniflow.model.server import ModelServerFactory
from uniflow.config import HuggingfaceConfig
from uniflow.model.config import HuggingfaceModelConfig
from uniflow.viz import Viz
from uniflow.schema import GuidedPrompt, Context

load_dotenv()

/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Prepare sample prompts

First, we need to demonstrate sample prompts for LLM, those include instruction and sample json format. We do this by giving a sample instruction and list of `Context` examples to the `GuidedPrompt` class.

In [ ]:
sample_instruction = """Generate one question and its corresponding answer based on the context. Following \
the format of the examples below to include context, question, and answer in the response."""

sample_examples = [
        Context(
            context="The quick brown fox jumps over the lazy dog.",
            question="What is the color of the fox?",
            answer="brown."
        ),
        Context(
            context="The quick brown fox jumps over the lazy black dog.",
            question="What is the color of the dog?",
            answer="black."
        )]

guided_prompt = GuidedPrompt(
    instruction=sample_instruction,
    examples=sample_examples
)

Second, we craft some dummy sample raw text strings. Below, we build a dataset with 400 text strings.

In [ ]:
raw_context_input = [
    """We believe our success depends upon our capabilities in areas such as design, research and development, \
production and marketing and is supported and protected by our intellectual property rights, such as \
trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy \
of applying for and registering intellectual property rights in the United States and select foreign countries \
on trademarks, inventions, innovations and designs that we deem valuable. W e also continue to vigorously \
protect our intellectual property, including trademarks, patents and trade secrets against third-party \
infringement and misappropriation.""",
    """In 1948, Claude E. Shannon published A Mathematical Theory of Communication (Shannon, 1948) \
establishing the theory of information. In his article, Shannon introduced the concept of information entropy \
for the first time. We will begin our journey here.""",
    """The chain rule states that the derivative of a composite function (a function composed of another \
function) is equal to the derivative of the outer function multiplied by the derivative of the inner function.\
Mathematically, it can be written as: \(\frac{d}{dx}g(h(x)) = \frac{dg}{dh}(h(x))\cdot \frac{dh}{dx}(x)\).""",
    """Hypothesis testing involves making a claim about a population parameter based on sample data, and then \
conducting a test to determine whether this claim is supported or rejected. This typically involves \
calculating a test statistic, determining a significance level, and comparing the calculated value to a \
critical value to obtain a p-value. """
]

raw_context_input_400 = raw_context_input * 100

print("sample size of raw context: ", len(raw_context_input_400))

sample size of raw context:  400


Next, for the given raw text strings `raw_context_input` above, we convert them to the `Context` class to be processed by `uniflow`.

In [ ]:

input_data = [
    Context(context=data)
    for data in raw_context_input_400
]

print("sample size of processed input data: ", len(input_data))

input_data[:2]


sample size of processed raw context with prompts:  400


[{'instruction': 'Generate one question and its corresponding answer based on the context. Following the format of the examples below to include context, question, and answer in the response.',
  'examples': [{'context': 'The quick brown fox jumps over the lazy dog.',
    'question': 'What is the color of the fox?',
    'answer': 'brown.'},
   {'context': 'The quick brown fox jumps over the lazy black dog.',
    'question': 'What is the color of the dog?',
    'answer': 'black.'},
   {'context': 'We believe our success depends upon our capabilities in areas such as design, research and development, production and marketing and is supported and protected by our intellectual property rights, such as trademarks, utility and design patents, copyrights, and trade secrets, among others. We have followed a policy of applying for and registering intellectual property rights in the United States and select foreign countries on trademarks, inventions, innovations and designs that we deem valuabl

### Use LLM to generate data

In this example, we will use the [HuggingfaceModelServer](https://github.com/CambioML/uniflow/blob/main/uniflow/model/server.py#L170)'s default LLM to generate questions and answers. Let's import the config and client of this model.

Here, we pass in our `guided_prompt` to the HuggingfaceConfig to use our customized instructions and examples, instead of the `uniflow` default ones.

Note, base on your GPU memory, you can set your optimal `batch_size` below. (We attached our `batch_size` benchmarking results in the appendix of this notebook.)

In [ ]:
config = HuggingfaceConfig(
    guided_prompt_template=guided_prompt,
    model_config=HuggingfaceModelConfig(batch_size=128))
client = Client(config)

/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.67s/it]


Now we call the `run` method on the `client` object to execute the question-answer generation operation on the data shown above.


In [ ]:
output = client.run(input_data)

  0%|          | 0/4 [00:00<?, ?it/s]

/opt/conda/envs/uniflow_no_installed/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:228: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
100%|██████████| 4/4 [03:01<00:00, 45.29s/it]


### Process the output

Let's take a look of the generation output. We need to do a little postprocessing on the raw output.

In [ ]:
import re
from pprint import pprint

keywords = ["context:", "question:", "answer:"]
pattern = '|'.join(map(re.escape, keywords))

o = output[0]['output'][0]['response'][0] ## we only postprocess the first output
segments = [segment for segment in re.split(pattern, o) if segment.strip()]
result = {
    "context": segments[-3],
    "question": segments[-2],
    "answer": segments[-1]
}

pprint(result, sort_dicts=False)

{'context': ' We believe our success depends upon our capabilities in areas '
            'such as design, research and development, production and '
            'marketing and is supported and protected by our intellectual '
            'property rights, such as trademarks, utility and design patents, '
            'copyrights, and trade secrets, among others. We have followed a '
            'policy of applying for and registering intellectual property '
            'rights in the United States and select foreign countries on '
            'trademarks, inventions, innovations and designs that we deem '
            'valuable. W e also continue to vigorously protect our '
            'intellectual property, including trademarks, patents and trade '
            'secrets against third-party \\ \n'
            'infringement and misappropriation.\n',
 'question': ' How does the company protect their intellectual property?\n',
 'answer': ' The company protects their intellectual property th

Congrats! Your question answers from the given knowledge context are generated!


---


## Appendix

We benchmarked to see the optimal `batch_size` for the `HuggingfaceModelConfig` flow. The answer is "It depends on your data token length, your GPU memory, your LLM size, etc." In the following experiment, we use a GPU with 24G memory and a quantized LLM (2G). We still use the above 400 raw data strings `raw_context_input_400`.


Here are the results:

- batch_size = 1
    100%|██████████| 400/400 [20:35<00:00,  3.09s/it]
- batch_size = 8
    100%|██████████| 50/50 [15:00<00:00, 18.02s/it]
- batch_size = 16
    100%|██████████| 25/25 [08:26<00:00, 20.25s/it]
- batch_size = 32
    100%|██████████| 13/13 [05:13<00:00, 24.09s/it]
- batch_size = 64
    100%|██████████| 7/7 [03:42<00:00, 31.75s/it]
- batch_size = 128
    100%|██████████| 4/4 [02:57<00:00, 44.33s/it]
- batch_size = 256: OOM

As you can see, the processing time is much shorter if `batch_size=128` compared with `batch_size=1`. However, it might lead to OOM error if `batch_size` is too large.